In [1]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.model_selection import cross_val_score

In [2]:
ames = pd.read_csv("AmesHousing.csv")
ames = ames.dropna(subset=['SalePrice', "Bldg Type", "Gr Liv Area", "TotRms AbvGrd"])
ames.head()



,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


In [6]:
lr = LinearRegression()
X = ames.drop('SalePrice', axis = 1)
y = ames["SalePrice"]
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [7]:
from sklearn.compose import ColumnTransformer

ct = ColumnTransformer(
  [
    ("standardize", StandardScaler(), ["Gr Liv Area", "TotRms AbvGrd"])
  ],
  remainder = "drop"
)


lr_pipeline1 = Pipeline(
  [("preprocessing", ct),
  ("linear_regression", LinearRegression())]
)

lr_pipeline1

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('standardize',
                                                  StandardScaler(),
                                                  ['Gr Liv Area',
                                                   'TotRms AbvGrd'])])),
                ('linear_regression', LinearRegression())])

Consider four possible models for predicting house prices:

Using only the size and number of rooms.

Using size, number of rooms, and building type.

Using size and building type, and their interaction.

Using a 5-degree polynomial on size, a 5-degree polynomial on number of rooms, and also building type.

Set up a pipeline for each of these four models.

Then, get predictions on the test set for each of your pipelines, and compute the root mean squared error. Which model performed best?

In [8]:
fitted1 = lr_pipeline1.fit(X_train, y_train)
pred1 = fitted1.predict(X_test)
rmse1 = sqrt(mean_squared_error(pred1,y_test))
rmse1

54066.98109699885

In [9]:
scores1 = cross_val_score(lr_pipeline1, X, y, cv=5, scoring='neg_mean_squared_error')
abs(scores1).mean()

3136138908.170903

In [10]:
ct = ColumnTransformer(
  [
    ("dummify", OneHotEncoder(sparse_output = False), ["Bldg Type"]),
    ("standardize", StandardScaler(), ["Gr Liv Area", "TotRms AbvGrd"])
  ],
  remainder = "drop"
)


lr_pipeline2 = Pipeline(
  [("preprocessing", ct),
  ("linear_regression", LinearRegression())]
)

lr_pipeline2

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('dummify',
                                                  OneHotEncoder(sparse_output=False),
                                                  ['Bldg Type']),
                                                 ('standardize',
                                                  StandardScaler(),
                                                  ['Gr Liv Area',
                                                   'TotRms AbvGrd'])])),
                ('linear_regression', LinearRegression())])

In [11]:
fitted2 = lr_pipeline2.fit(X_train, y_train)
pred2 = fitted2.predict(X_test)
rmse2 = sqrt(mean_squared_error(pred2,y_test))
rmse2

52035.41493383295

Before doing cross validation, Model 2 has the lowest RMSE.

In [12]:
scores2 = cross_val_score(lr_pipeline2, X, y, cv=5, scoring='neg_mean_squared_error')
abs(scores2).mean()

2951993958.10073

After doing cross validation, model 2 has the lowest average MSE which is what we got when initially finding our RMSE with the initial train_test_split.

In [14]:
ct_dummies = ColumnTransformer(
  [("dummify", OneHotEncoder(sparse_output = False), ["Bldg Type"])],
  remainder = "passthrough"
).set_output(transform = "pandas")

ct_inter = ColumnTransformer(
  [
    ("interaction", PolynomialFeatures(interaction_only = True), ["remainder__Gr Liv Area", "dummify__Bldg Type_1Fam"]),
  ],
  remainder = "drop"
).set_output(transform = "pandas")

lr_pipeline3 = Pipeline(
  [("preprocessing1", ct_dummies),
   ("preprocessing2", ct_inter),
  ("linear_regression", LinearRegression())]
)

In [15]:
fitted3 = lr_pipeline3.fit(X_train, y_train)
pred3 = fitted3.predict(X_test)
rmse3 = sqrt(mean_squared_error(pred3,y_test))
rmse3

54362.9441738997

In [16]:
scores3 = cross_val_score(lr_pipeline3, X, y, cv=5, scoring='neg_mean_squared_error')
abs(scores3).mean()

3132206753.875458

In [17]:
ct = ColumnTransformer(
    [("dummify", OneHotEncoder(sparse_output=False), ["Bldg Type"]),
     ("degreesize", PolynomialFeatures(degree=5, interaction_only=False), ["Gr Liv Area"]),
     ("degreesize2", PolynomialFeatures(degree=5, interaction_only=False), ["TotRms AbvGrd"])],
    remainder="drop"
).set_output(transform = "pandas")

lr_pipeline4 = Pipeline(
  [("preprocessing1", ct),
  ("linear_regression", LinearRegression())]
)
lr_pipeline4

Pipeline(steps=[('preprocessing1',
                 ColumnTransformer(transformers=[('dummify',
                                                  OneHotEncoder(sparse_output=False),
                                                  ['Bldg Type']),
                                                 ('degreesize',
                                                  PolynomialFeatures(degree=5),
                                                  ['Gr Liv Area']),
                                                 ('degreesize2',
                                                  PolynomialFeatures(degree=5),
                                                  ['TotRms AbvGrd'])])),
                ('linear_regression', LinearRegression())])

In [18]:
ct.fit_transform(X_train)

,dummify__Bldg Type_1Fam,dummify__Bldg Type_2fmCon,dummify__Bldg Type_Duplex,dummify__Bldg Type_Twnhs,dummify__Bldg Type_TwnhsE,degreesize__1,degreesize__Gr Liv Area,degreesize__Gr Liv Area^2,degreesize__Gr Liv Area^3,degreesize__Gr Liv Area^4,degreesize__Gr Liv Area^5,degreesize2__1,degreesize2__TotRms AbvGrd,degreesize2__TotRms AbvGrd^2,degreesize2__TotRms AbvGrd^3,degreesize2__TotRms AbvGrd^4,degreesize2__TotRms AbvGrd^5
1802,1.0,0.0,0.0,0.0,0.0,1.0,1584.0,2509056.0,3.974345e+09,6.295362e+12,9.971853e+15,1.0,6.0,36.0,216.0,1296.0,7776.0
2154,1.0,0.0,0.0,0.0,0.0,1.0,1668.0,2782224.0,4.640750e+09,7.740770e+12,1.291161e+16,1.0,7.0,49.0,343.0,2401.0,16807.0
1848,0.0,0.0,0.0,1.0,0.0,1.0,1441.0,2076481.0,2.992209e+09,4.311773e+12,6.213265e+15,1.0,5.0,25.0,125.0,625.0,3125.0
99,1.0,0.0,0.0,0.0,0.0,1.0,1478.0,2184484.0,3.228667e+09,4.771970e+12,7.052972e+15,1.0,7.0,49.0,343.0,2401.0,16807.0
2723,1.0,0.0,0.0,0.0,0.0,1.0,960.0,921600.0,8.847360e+08,8.493466e+11,8.153727e+14,1.0,3.0,9.0,27.0,81.0,243.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1434,1.0,0.0,0.0,0.0,0.0,1.0,1418.0,2010724.0,2.851207e+09,4.043011e+12,5.732990e+15,1.0,5.0,25.0,125.0,625.0,3125.0
1562,0.0,0.0,0.0,0.0,1.0,1.0,1646.0,2709316.0,4.459534e+09,7.340393e+12,1.208229e+16,1.0,5.0,25.0,125.0,625.0,3125.0
696,1.0,0.0,0.0,0.0,0.0,1.0,1306.0,1705636.0,2.227561e+09,2.909194e+12,3.799408e+15,1.0,6.0,36.0,216.0,1296.0,7776.0
4,1.0,0.0,0.0,0.0,0.0,1.0,1629.0,2653641.0,4.322781e+09,7.041811e+12,1.147111e+16,1.0,6.0,36.0,216.0,1296.0,7776.0


In [19]:
fitted4 = lr_pipeline4.fit(X_train, y_train)
pred4 = fitted4.predict(X_test)
rmse4 = sqrt(mean_squared_error(pred4,y_test))
rmse4

53670.62008164176

In [20]:
scores4 = cross_val_score(lr_pipeline4, X, y, cv=5, scoring='neg_mean_squared_error')
abs(scores4).mean()

3198787103.668873

In [39]:
cross_val_score(lr_pipeline4, X, y, cv=5, scoring='r2').mean()

0.49269447888157974

In [47]:
ct = ColumnTransformer(
    [("dummify", OneHotEncoder(sparse_output=False), ["Bldg Type"]),
     ("degreesize", PolynomialFeatures(), ["Gr Liv Area"]),
     ("degreesize2", PolynomialFeatures(), ["TotRms AbvGrd"])],
    remainder="drop"
).set_output(transform = "pandas")

lr_pipeline6 = Pipeline(
  [("preprocessing1", ct),
  ("linear_regression", LinearRegression())]
)

EN = {'preprocessing1__degreesize__degree': np.arange(1,11), "preprocessing1__degreesize2__degree": np.arange(1,11)}

gscv6 = GridSearchCV(lr_pipeline6, EN, cv = 5, scoring='r2')

In [48]:
gscv_fitted6 = gscv6.fit(X_train, y_train)

In [49]:
gscv_fitted6.cv_results_['mean_test_score']

array([ 0.53815314,  0.54625082,  0.55971742,  0.55410904,  0.48374902,
        0.41829969,  0.18965182, -0.48769278, -2.14995258, -5.94949081,
        0.53634266,  0.54339543,  0.56068916,  0.55618709,  0.48374902,
        0.41829969,  0.18965182, -0.48769278, -2.14995258, -5.9494908 ,
        0.54013664,  0.5467162 ,  0.56026655,  0.55628174,  0.52601561,
        0.41829969,  0.18965182, -0.48769278, -2.14995258, -5.9494908 ,
        0.54887038,  0.55411264,  0.56507535,  0.56178103,  0.52229568,
        0.41829969,  0.18965182, -0.48769278, -2.14995258, -5.94949154,
        0.54933086,  0.55353083,  0.56208974,  0.56153682,  0.51804385,
        0.4182997 ,  0.18965182, -0.48769278, -2.14995258, -5.94949154,
        0.54438365,  0.55199646,  0.56043555,  0.55701544,  0.52871184,
        0.51785179,  0.18965182, -0.48769278, -2.14995258, -5.94949154,
        0.52569457,  0.54412534,  0.56354989,  0.55184995,  0.52540498,
        0.5172945 ,  0.18965182, -0.48769278, -2.14995258, -5.94

In [50]:
gscv_fitted6.best_params_

{'preprocessing1__degreesize2__degree': 4,
 'preprocessing1__degreesize__degree': 3}

In [59]:
results_6 = gscv_fitted6.cv_results_

params_df = pd.DataFrame(results_6)
params_df = df[['param_preprocessing1__degreesize__degree',
         'param_preprocessing1__degreesize2__degree',
         'mean_test_score']]
print(gscv_fitted6.best_params_)
params_df

{'preprocessing1__degreesize2__degree': 4, 'preprocessing1__degreesize__degree': 3}


,param_preprocessing1__degreesize__degree,param_preprocessing1__degreesize2__degree,mean_test_score
0,1,1,0.538153
1,2,1,0.546251
2,3,1,0.559717
3,4,1,0.554109
4,5,1,0.483749
...,...,...,...
76,5,9,0.495644
77,6,9,0.415390
78,7,9,0.425050
79,8,9,-0.487693


In [71]:
ct = ColumnTransformer(
    [("dummify", OneHotEncoder(sparse_output=False), ["Bldg Type"]),
     ("degreesize", PolynomialFeatures((1,3)), ["Gr Liv Area"]),
     ("degreesize2", PolynomialFeatures((1,4)), ["TotRms AbvGrd"])],
    remainder="drop"
).set_output(transform = "pandas")

lr_pipeline7 = Pipeline(
  [("preprocessing1", ct),
  ("linear_regression", LinearRegression())]
)


In [72]:
fitted7 = lr_pipeline7.fit(X_train, y_train)
pred7 = fitted7.predict(X_test)
rmse7 = sqrt(mean_squared_error(pred7,y_test))
rmse7

51684.56208497074

In [73]:
scores7 = cross_val_score(lr_pipeline3, X, y, cv=5, scoring='r2')
scores7.mean()

0.5028425816120953

In [74]:
lr_pipeline3.named_steps['linear_regression'].coef_

array([ 0.00000000e+00,  5.59640202e+01, -7.34788465e+04,  5.96722505e+01])

Q1: The best model was the one that used a degree 3 polynomial for Gr Liv Area and a degree four polynomial for TotRms AvgGrd. The rmse was 51684 and the r squared average score was .502.

Q2: The downsides of trying all possible models is that it takes a lot of human time so run all of the models. It is also very inefficient and the run time increases. We could also run into some overfitting issues.